<a href="https://colab.research.google.com/github/Onmang/Multimedia_engineering/blob/master/T20ME022_ex07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip "/content/drive/MyDrive/Colab Notebooks/resistor_v3.zip"

In [2]:
# ライブラリのインポート
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.models import Model
import numpy as np
import cv2  
import matplotlib.pyplot as plt
import os # ディレクトリ（フォルダ）やファイルを扱うためのライブラリ（本当はPathlibライブラリのほうが良いが難しいので簡単な方で）
import glob # ファイル一覧を取得するためのライブラリ
import re # 正規表現を使ったパターンマッチング用（ラベルを取得するため）
tf.test.gpu_device_name() # GPUの利用確認

'/device:GPU:0'

In [3]:
train_list = glob.glob('data/train/*/*.png') # 訓練用画像ファイルの取得（拡張子がpng）
valid_list = glob.glob('data/valid/*/*.png') # 検証用画像ファイルの取得
classes = sorted(os.listdir('data/train'), key=int) # 教師ラベルの一覧をリストで取得する。"sorted"でソートしておく。
print(classes) # 取得した抵抗器のラベルを表示。文字列になっている点に注意すること！

['150', '160', '390', '430', '620', '1600', '1800', '2200', '2400', '3000', '3300', '3600', '5600', '9100']


In [4]:
train_ds = tf.data.Dataset.list_files(train_list) # 訓練ファイル名のリストをTensor型に変換
valid_ds = tf.data.Dataset.list_files(valid_list) # テストファイル名のリストをTensor型に変換

In [5]:
train_ds = tf.data.Dataset.list_files(train_list) # 訓練ファイル名のリストをTensor型に変換
valid_ds = tf.data.Dataset.list_files(valid_list) # 検証用のファイル名のリストをTensor型に変換

# ファイル名から画像データをロードしてNNへ入力できるようにデータを成形する。ついでに教師ラベルも取得する
def load_file(files):
    ys = [] # ラベル
    xs = [] # 入力データ
    for f in files:        
        file = bytes.decode(f.numpy()) # ファイル名はTensor型で保存されているため，文字列型として取得する。
        m = re.search(r'/(\d+)_', file) # 正規表現を使ってファイル名から抵抗値を取得する。
        label = m.groups()[0] # 抵抗値を取得しlabelに保存
        ys.append(classes.index(label)) # ラベルをラベルIDに変換してラベルリストに追加する
        #print(f'resistor = {label}, index={classes.index(label)}')
        img = cv2.imread(file) # 画像データをカラーで取得。画像サイズは64x64になっているのでここではリサイズしない。
        xs.append(img) # データを入力データリストに追加
    xs = np.array(xs, dtype=np.float32) / 255. # 正規化してfloat32の行列に変換する
    ys = np.array(ys, dtype=np.int32) # ラベルも行列に変換
    return xs, ys

#
# tf.Dataによるtf.Tensor変換      
#
AUTOTUNE = tf.data.experimental.AUTOTUNE # 処理を最適化するためのおまじない（自動チューニング設定）
train_ds = train_ds.shuffle(len(train_list)) # 訓練データをシャッフルする。引数にはデータ数を指定すると完全なシャッフルが行われる。len(x_train)は60000。
train_ds = train_ds.repeat(1) # 1 epochで使われるデータの回数。1の場合，1epochで1回しか使われない。引数を空欄にすると無限に使われる。
train_ds = train_ds.batch(32) # ミニバッチを作る。1バッチ32個のデータ。
train_ds = train_ds.map(lambda files: tf.py_function(load_file, [files], Tout=[tf.float32, tf.int32])) # ファイル名から入力ラベルとラベルを取得
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE) # 訓練中に次のバッチを取り出すための処理。

valid_ds = valid_ds.batch(32) # 検証データはシャッフルする必要ないので，バッチ化のみの処理でOK。
valid_ds = valid_ds.map(lambda x: tf.py_function(load_file, [x], Tout=[tf.float32, tf.int32]))

In [6]:
# Functional API
input = Input(shape=(64, 64, 3), name='input') # 入力層の定義　64×64×3 （カラー画像）
h = Conv2D(16, (3, 3), padding='same', activation='relu', name='cnn01')(input)
h = MaxPooling2D((2, 2), name='pool01')(h)
h = Conv2D(32, (3, 3), padding='same', activation='relu', name='cnn02')(h)
h = MaxPooling2D((2, 2), name='pool02')(h)
h = Conv2D(64, (3, 3), padding='same', activation='relu', name='cnn03')(h)
h = MaxPooling2D((2, 2), name='pool03')(h)
h = Conv2D(128, (3, 3), padding='same', activation='relu', name='cnn04')(h)
h = MaxPooling2D((2, 2), name='pool04')(h)
h = Conv2D(256, (3, 3), padding='valid', activation='relu', name='cnn05')(h)
h = MaxPooling2D((2, 2), name='pool05')(h)
h = Flatten(name='flatten')(h) # GlobalAveragePoolingでも良い
h = Dense(128, activation='relu', name='dense01')(h) # 全結合層の隠れ層のノードは128
output = Dense(len(classes), activation='softmax', name='output')(h) # 出力層

model = Model(inputs=input, outputs=output) # この処理でモデルを実体化する。入力層と出力層を渡すと自動的にネットワークを構築してくれる。
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 64, 64, 3)]       0         
                                                                 
 cnn01 (Conv2D)              (None, 64, 64, 16)        448       
                                                                 
 pool01 (MaxPooling2D)       (None, 32, 32, 16)        0         
                                                                 
 cnn02 (Conv2D)              (None, 32, 32, 32)        4640      
                                                                 
 pool02 (MaxPooling2D)       (None, 16, 16, 32)        0         
                                                                 
 cnn03 (Conv2D)              (None, 16, 16, 64)        18496     
                                                                 
 pool03 (MaxPooling2D)       (None, 8, 8, 64)          0     

In [7]:
# TFのバグでこのように書く
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
#model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 訓練の実施
model.fit(train_ds, epochs=10, validation_data=valid_ds)

Epoch 1/10
104/104 [==============================] - 18s 35ms/step - loss: 2.0099 - sparse_categorical_accuracy: 0.2871 - val_loss: 1.2282 - val_sparse_categorical_accuracy: 0.5143
Epoch 2/10
104/104 [==============================] - 3s 29ms/step - loss: 1.0111 - sparse_categorical_accuracy: 0.6185 - val_loss: 0.7560 - val_sparse_categorical_accuracy: 0.7039
Epoch 3/10
104/104 [==============================] - 3s 28ms/step - loss: 0.3806 - sparse_categorical_accuracy: 0.8759 - val_loss: 0.1281 - val_sparse_categorical_accuracy: 0.9662
Epoch 4/10
104/104 [==============================] - 4s 37ms/step - loss: 0.1394 - sparse_categorical_accuracy: 0.9593 - val_loss: 0.0641 - val_sparse_categorical_accuracy: 0.9922
Epoch 5/10
104/104 [==============================] - 3s 30ms/step - loss: 0.0774 - sparse_categorical_accuracy: 0.9786 - val_loss: 0.0276 - val_sparse_categorical_accuracy: 0.9922
Epoch 6/10
104/104 [==============================] - 3s 28ms/step - loss: 0.0816 - sparse_cat

In [8]:
# 推論のテスト
img = cv2.imread('data/valid/3000/3000_200_0.png')
img = img.reshape(1, 64, 64, 3)
img = np.float32(img) / 255.
pred = model.predict(img)
print(f'{pred.argmax()} --> {classes[pred.argmax()]} ohm')

1/1 [==============================] - 0s 117ms/step
9 --> 3000 ohm


ここから、回帰モデル

In [41]:
train_ds = tf.data.Dataset.list_files(train_list) # 訓練ファイル名のリストをTensor型に変換
valid_ds = tf.data.Dataset.list_files(valid_list) # 検証用のファイル名のリストをTensor型に変換

# ファイル名から画像データをロードしてNNへ入力できるようにデータを成形する。ついでに教師ラベルも取得する
def load_file(files):
    ys = [] # ラベル
    xs = [] # 入力データ
    for f in files:        
        file = bytes.decode(f.numpy()) # ファイル名はTensor型で保存されているため，文字列型として取得する。
        m = re.search(r'/(\d+)_', file) # 正規表現を使ってファイル名から抵抗値を取得する。
        label = m.groups()[0] # 抵抗値を取得しlabelに保存
        ys.append(label) # ラベルをラベルリストに追加する
        img = cv2.imread(file) # 画像データをカラーで取得。画像サイズは64x64になっているのでここではリサイズしない。
        xs.append(img) # データを入力データリストに追加
    xs = np.array(xs, dtype=np.float32) / 255. # 正規化してfloat32の行列に変換する
    ys = np.array(ys, dtype=np.float32) # ラベルも行列に変換
    return xs, ys

#
# tf.Dataによるtf.Tensor変換      
#
AUTOTUNE = tf.data.experimental.AUTOTUNE # 処理を最適化するためのおまじない（自動チューニング設定）
train_ds = train_ds.shuffle(len(train_list)) # 訓練データをシャッフルする。引数にはデータ数を指定すると完全なシャッフルが行われる。len(x_train)は60000。
train_ds = train_ds.repeat(1) # 1 epochで使われるデータの回数。1の場合，1epochで1回しか使われない。引数を空欄にすると無限に使われる。
train_ds = train_ds.batch(32) # ミニバッチを作る。1バッチ32個のデータ。
train_ds = train_ds.map(lambda files: tf.py_function(load_file, [files], Tout=[tf.float32, tf.float32])) # ファイル名から入力ラベルとラベルを取得
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE) # 訓練中に次のバッチを取り出すための処理。

valid_ds = valid_ds.batch(32) # 検証データはシャッフルする必要ないので，バッチ化のみの処理でOK。
valid_ds = valid_ds.map(lambda x: tf.py_function(load_file, [x], Tout=[tf.float32, tf.float32]))

In [42]:
# Functional API
input = Input(shape=(64, 64, 3), name='input') # 入力層の定義　64×64×3 （カラー画像）
h = Conv2D(16, (3, 3), padding='same', activation='relu', name='cnn01')(input)
h = MaxPooling2D((2, 2), name='pool01')(h)
h = Conv2D(32, (3, 3), padding='same', activation='relu', name='cnn02')(h)
h = MaxPooling2D((2, 2), name='pool02')(h)
h = Conv2D(64, (3, 3), padding='same', activation='relu', name='cnn03')(h)
h = MaxPooling2D((2, 2), name='pool03')(h)
h = Conv2D(128, (3, 3), padding='same', activation='relu', name='cnn04')(h)
h = MaxPooling2D((2, 2), name='pool04')(h)
h = Conv2D(256, (3, 3), padding='valid', activation='relu', name='cnn05')(h)
h = MaxPooling2D((2, 2), name='pool05')(h)
h = Flatten(name='flatten')(h) # GlobalAveragePoolingでも良い
h = Dense(128, activation='relu', name='dense01')(h) # 全結合層の隠れ層のノードは128
output = Dense(1, activation='linear', name='output')(h) # 出力層

model = Model(inputs=input, outputs=output) # この処理でモデルを実体化する。入力層と出力層を渡すと自動的にネットワークを構築してくれる。
model.summary()

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 64, 64, 3)]       0         
                                                                 
 cnn01 (Conv2D)              (None, 64, 64, 16)        448       
                                                                 
 pool01 (MaxPooling2D)       (None, 32, 32, 16)        0         
                                                                 
 cnn02 (Conv2D)              (None, 32, 32, 32)        4640      
                                                                 
 pool02 (MaxPooling2D)       (None, 16, 16, 32)        0         
                                                                 
 cnn03 (Conv2D)              (None, 16, 16, 64)        18496     
                                                                 
 pool03 (MaxPooling2D)       (None, 8, 8, 64)          0   

In [51]:
# TFのバグでこのように書く
#model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
model.compile(optimizer='adam', loss='mse', metrics=['mae', 'mse'])
#model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 訓練の実施
model.fit(train_ds, epochs=110, validation_data=valid_ds)

Epoch 1/110
104/104 [==============================] - 8s 45ms/step - loss: 3330.6206 - mae: 31.3082 - mse: 3316.5117 - val_loss: 27042.0078 - val_mae: 46.8574 - val_mse: 25028.1152
Epoch 2/110
104/104 [==============================] - 3s 30ms/step - loss: 1584.1007 - mae: 24.5186 - mse: 1602.5001 - val_loss: 26782.3145 - val_mae: 63.3818 - val_mse: 24797.7812
Epoch 3/110
104/104 [==============================] - 4s 34ms/step - loss: 2398.0847 - mae: 30.8274 - mse: 2394.8215 - val_loss: 42469.4102 - val_mae: 103.4877 - val_mse: 39639.3359
Epoch 4/110
104/104 [==============================] - 4s 38ms/step - loss: 4570.5547 - mae: 38.1755 - mse: 4550.9336 - val_loss: 20833.7852 - val_mae: 45.3009 - val_mse: 19281.3887
Epoch 5/110
104/104 [==============================] - 3s 30ms/step - loss: 1914.2922 - mae: 27.3913 - mse: 1912.7546 - val_loss: 24140.7852 - val_mae: 52.2745 - val_mse: 22343.4082
Epoch 6/110
104/104 [==============================] - 3s 31ms/step - loss: 2459.2856 - m

In [52]:
# 推論のテスト
img = cv2.imread('data/valid/3000/3000_200_0.png')
img = img.reshape(1, 64, 64, 3)
img = np.float32(img) / 255.
pred = model.predict(img)
print(f'{classes[pred.argmax()]} ohm')

1/1 [==============================] - 0s 94ms/step
150 ohm
